In [2]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import gridspec

from scipy.interpolate import InterpolatedUnivariateSpline

from matplotlib import figure

import ipywidgets as widgets
from ipywidgets import interact, interactive
from ipywidgets import HBox, VBox, Box
from ipywidgets import GridspecLayout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, Text, ButtonStyle

from operator import itemgetter

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))



In [3]:
SNeff_path = 'MDet_yrs_vs_SNeff.csv'
SNeff_file = open(SNeff_path, "r")

MDet_years1 = []
SNeffs1 = []

lines = SNeff_file.readlines()
for line in lines[6:36]:
    vals = line.split(',')
    MDet_years1.append(float(vals[2]))
    SNeffs1.append(float(vals[3]))
    

MDet_to_SNeff = InterpolatedUnivariateSpline(MDet_years1, SNeffs1)

In [4]:
Freeze_path = 'Freeze_outs.csv'
Freeze_file = open(Freeze_path, "r")

SNeffs2 = []
spin0 = []
spin05 = []
spin1 = []

lines = Freeze_file.readlines()
for line in lines[10:]:
    vals = line.split(',')
    SNeffs2.append(float(vals[0]))
    
    spin0.append(float(vals[1]))
    spin05.append(float(vals[2]) if len(vals[2]) > 0 else np.nan)
    spin1.append(float(vals[3]) if len(vals[3]) > 0 else np.nan)

SNeff_to_spin0 = InterpolatedUnivariateSpline(SNeffs2, spin0)
SNeff_to_spin05 = InterpolatedUnivariateSpline(SNeffs2[20:], spin05[20:])
SNeff_to_spin1 = InterpolatedUnivariateSpline(SNeffs2[27:], spin1[27:])

In [8]:
def make_plot(SO_eff,
              
              SO_Used, 
              SO_Start,
              
              AdvSO_Used, 
              AdvSO_Start, 
              AdvSO_End,
              
              CHLAT1_eff,
              
              CHLAT1_Used, 
              CHLAT1_Start, 
              CHLAT1_End,
              
              CHALT2_Used, 
              CHLAT2_Start, 
              CHLAT2_End,
                 
              y_axis,
              Update):
    
    if AdvSO_Used:
        SO_Duration = AdvSO_Start - SO_Start
    
    Detectors    = [0.031, 0.062  , 0.135    , 0.135    ] 
    Efficiencies = [SO_eff , SO_eff , CHLAT1_eff , CHLAT1_eff ]
    Names        = ["SO" , "AdvSO", "CHLAT 1", "CHLAT 2"]
    
    SO_End = AdvSO_Start
    if AdvSO_Used == False:
        SO_End = AdvSO_End
        
    StartYears   = [SO_Start, AdvSO_Start,  CHLAT1_Start,  CHLAT2_Start]
    EndYears     = [SO_End,  AdvSO_End,  CHLAT1_End,  CHLAT2_End]
    Used         = [SO_Used,  AdvSO_Used,  CHLAT1_Used,  CHALT2_Used]
    
    increment = 1/52
        
    if Used[0] or Used[1] or Used[2] or Used[3]:
        used_map = map(int, Used)
        used_int = list(used_map)
        st_years = np.multiply(StartYears, used_int)
        st_years = st_years[st_years != 0]
        FirstYear = np.min(st_years)
        LastYear = np.max(np.multiply(EndYears, used_int))

       

        years = np.arange(FirstYear, LastYear, increment)
        det_times = []
        total_det_time = 0

        for date in years:
            for i in range(len(Detectors)):
                if Used[i]:
                    if date >= StartYears[i] and date <= EndYears[i]:
                        total_det_time += Detectors[i]*Efficiencies[i]*increment

            det_times.append(total_det_time)

        These_SNeffs = MDet_to_SNeff(det_times)

        These_spin0s = SNeff_to_spin0(These_SNeffs)
        These_spin05s = SNeff_to_spin05(These_SNeffs)
        These_spin1s = SNeff_to_spin1(These_SNeffs)
    


    #===============================================================================================
    
    StartYears_2   = [2024, 2029,  2029,  2029]
    Durations_2    = [5,  10,  7,  7]
    Used_2         = [False,  False,  True,  True]
    Efficiencies_2 = [0.9, 0.9, 1.0, 1.0]

    used_map_2 = map(int, Used_2)
    used_int_2 = list(used_map_2)
    st_years_2 = np.multiply(StartYears_2, used_int_2)
    st_years_2 = st_years_2[st_years_2 != 0]
    FirstYear_2 = np.min(st_years_2)
    LastYear_2 = np.max(np.multiply(np.add(StartYears_2,Durations_2), used_int_2))

    years_2 = np.arange(FirstYear_2, LastYear_2, increment)
    det_times_2 = []
    total_det_time_2 = 0

    for date_2 in years_2:
        for i in range(len(Detectors)):
            if Used_2[i]:
                if date_2 >= StartYears_2[i] and date_2 <= StartYears_2[i] + Durations_2[i]:
                    total_det_time_2 += Detectors[i]*Efficiencies_2[i]*increment

        det_times_2.append(total_det_time_2)

    These_SNeffs_2 = MDet_to_SNeff(det_times_2)

    These_spin0s_2 = SNeff_to_spin0(These_SNeffs_2)
    These_spin05s_2 = SNeff_to_spin05(These_SNeffs_2)
    These_spin1s_2 = SNeff_to_spin1(These_SNeffs_2)


    
    #===============================================================================================


    %matplotlib inline

    fig = plt.figure(figsize=(15, 10), dpi=80, constrained_layout=False)
    #fig.suptitle('Time-series Comparason', fontsize=24)
    gs1 = fig.add_gridspec(nrows = 1, ncols = 1, wspace=0.001) 

    ax0 = plt.subplot(gs1[0, 0])
    
    labels = False
    
    if(y_axis == 'Freeze Out'):
        anot = 300 #max(These_spin1s_2[-1], These_spin1s[-1])
        ax0.plot((2022, 2041.8), (anot, anot), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)) )
        ax0.text(2041.9, anot, "QCD Phase Transition", c='#bfbfbf', fontsize=15, transform=ax0.transData)
        
        if Used[0] or Used[1] or Used[2] or Used[3]:
            ax0.plot(years[:], These_spin0s[:], color='blue', linestyle='dotted')
            ax0.plot(years[:], These_spin05s[:], color='blue', linestyle='dashed')
            ax0.plot(years[:], These_spin1s[:], label='Custom', color='blue')

        ax0.plot(years_2[:], These_spin0s_2[:], color='red', linestyle='dotted')
        ax0.plot(years_2[:], These_spin05s_2[:], color='red', linestyle='dashed')
        ax0.plot(years_2[:], These_spin1s_2[:], label='CMB-S4 PBD', color='red')

        ax0.plot([2022,2023,2024], [1,2,3], c = "#bfbfbf", label='Spin 1')
        ax0.plot([2022,2023,2024], [1,2,3], c = "#bfbfbf", linestyle='dashed', label='Spin 1/2')
        ax0.plot([2022,2023,2024], [1,2,3], c = "#bfbfbf", linestyle='dotted', label='Spin 0')

        if labels:
            for i in range(len(Names)):
                if Used[i]:
                    x = StartYears[i]
                    year_i = int((x-FirstYear)/increment)
                    y = These_spin1s[year_i]
                    ax0.annotate(Names[i], xy=(x, y), xytext=(0.01, 0.5+0.07*i), textcoords='axes fraction', fontsize=15, arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=0,angleB=-90,rad=5.0"))

    if(y_axis == 'Sigma(N_eff)'):
        anot = 0.030
        ax0.plot((2022, 2041.8), (anot, anot), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)) )
        ax0.text(2041.9, anot, "CMB-S4 Target", c='#bfbfbf', fontsize=15, transform=ax0.transData)
        anot1 = 0.035
        #ax0.plot((2022, 2041.8), (anot1, anot1), scaley = False, c = "#bfbfbf", linestyle= (0, (3, 5, 1, 5)) )
        #ax0.text(2041.9, anot1, "QCD Phase Transition", c='#bfbfbf', fontsize=15, transform=ax0.transData)
        
        if Used[0] or Used[1] or Used[2] or Used[3]:
            ax0.plot(years[:], These_SNeffs[:], label='Custom', color='blue')
            
        ax0.plot(years_2[:], These_SNeffs_2[:], label='CMB-S4 PBD', color='red')

        if(labels):
            for i in range(len(Names)):
                if Used[i]:
                    x = StartYears[i]
                    year_i = int((x-FirstYear)/increment)
                    y = These_SNeffs[year_i]
                    ax0.annotate(Names[i], xy=(x, y), xytext=(0.01, 0.6-0.07*i), textcoords='axes fraction', fontsize=15, arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=0,angleB=90,rad=5.0"))

        
    #for i in range(len(StartYears)):
        #if(Used[i]):
            #ax0.plot((StartYears[i], StartYears[i]), (100, 10000), scaley = False, c = "#878787")
            #ax0.text(StartYears[i] - 0.25, 12000, Names[i], rotation = 90, fontsize = 23, c = "#878787")

    

    ax0.set_xlabel('Year', fontsize = 30)
   
    ax0.tick_params(axis='x', labelsize=23)
    ax0.tick_params(axis='y', labelsize=23)
    ax0.set_xlim(2022,2047)
    if(y_axis == 'Freeze Out'):
        ax0.set_ylabel('Freeze Out Temp [MeV]', fontsize = 30)
        ax0.set_ylim(80,300000)
        ax0.set_yscale('log')
    if(y_axis == 'Sigma(N_eff)'):
        ax0.set_ylabel('Sigma(N_eff)', fontsize = 30)
        ax0.set_ylim(0.025,0.05)
        
    
    ax0.legend()

    # fig.tight_layout()
    #fig.savefig("Test-Timeline-Graph.png")
    plt.show()
    
    
    
    
w = interactive(make_plot, 
              
              SO_eff            = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description=' ' , indent=False, readout_format=".0%"),

              SO_Used           = widgets.Checkbox(value=True, description='Baseline', indent=False), 
              SO_Start          = widgets.Dropdown(options = np.arange(2024,2030,1), value=2024, description='Start Year:'),
                
              AdvSO_Used        = widgets.Checkbox(value=True, description='Advanced', indent=False) , 
              AdvSO_Start       = widgets.Dropdown(options = np.arange(2029,2035,1), value=2029, description='Start Year:'),
              AdvSO_End         = widgets.Dropdown(options = np.arange(2029,2046,1), value=2036, description='End Year:'),
              
              CHLAT1_eff        = widgets.FloatSlider(value=1.0, min=0.5, max=1.0, step=0.05, description=' ' , indent=False, readout_format=".0%"),
             
              CHLAT1_Used       = widgets.Checkbox(value=True, description='CHLAT #1', indent=False) , 
              CHLAT1_Start      = widgets.Dropdown(options = np.arange(2029,2035,1), value=2029, description='Start Year:'),
              CHLAT1_End        = widgets.Dropdown(options = np.arange(2030,2046,1), value=2036, description='End Year:'),

              CHALT2_Used       = widgets.Checkbox(value=False, description='CHLAT #2', indent=False), 
              CHLAT2_Start      = widgets.Dropdown(options = np.arange(2029,2035,1), value=2029, description='Start Year:'), 
              CHLAT2_End        = widgets.Dropdown(options = np.arange(2030,2046,1), value=2036, description='End Year:'),

              y_axis            = widgets.Dropdown(options=['Freeze Out', 'Sigma(N_eff)'], value='Freeze Out', description='Y Axis:', disabled=False), 
                
              Update            = widgets.ToggleButton(value=False, description='Run'))


w.children

# This doesn't display the plot until one of the widgets has been used
#display(HBox([VBox(list(itemgetter(0,-1,-2)(w.children))), VBox(w.children[1:-2])]))
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='3px solid',
                    width='100%')

spacer = Layout(height='30px')
space = Layout(width='30px')

sim_text = "Simons Observatory:"
sim_lab = widgets.HTML(value = f"<b><font size='4px'>{sim_text}</b>")
sim_box = VBox(w.children[1:3])

adv_sim_box = VBox(w.children[3:6])

eff_text = "Relative Efficiency (Per Detector):"
eff_lab = widgets.HTML(value = f"<b><font size='2px'>{eff_text}</b>")
                       
SO_eff = w.children[0]
                       
b1 = VBox([sim_lab, sim_box, adv_sim_box, eff_lab, SO_eff], layout=box_layout)

s1 = Box([], layout=spacer)

CMB_text = "CMB-S4:"
chlat_1_lab = widgets.HTML(value = f"<b><font size='4px'>{CMB_text}</b>")
chlat_1_box = VBox(w.children[7:10])
b2 = VBox([chlat_1_lab, chlat_1_box])

s2 = Box([], layout=spacer)


chlat_2_box = VBox(w.children[10:13])

con_text = "Construction Fraction:"
con_lab = widgets.HTML(value = f"<b><font size='2px'>{con_text}</b>")
                       
con = w.children[6]

CMB_box = VBox([b2, s2, chlat_2_box, con_lab, con], layout=box_layout)

col_1 = VBox([b1, s1, CMB_box])

sp = Box([], layout=space)

col_2 = VBox(list(itemgetter(-3, -1, -2)(w.children)))

window = HBox([col_1, sp, col_2])

Title_text = "Light Relic Science With Time:"
title = widgets.HTML(value = f"<b><font size='6px'>{Title_text}</b>")

sub_text = "This plot compares the light relic science achieved as a function of time by the CMB-S4 Preliminary Baseline Design with custom configurations assembled from the Simons Observatory Baseline and Advanced and the CMB-S4 Chilean Large Aperture Telescopes.  The achieved science can be represented as the constraint on the effective number of light relic particles (N_eff) or the freeze-out temperature of additional spin-0, -1/2, and -1 particles. Each telescope can be toggled on or off, and have its start and end date adjusted. Each experiment can also be rescaled for its relative efficiency (SO) or fraction of the PBD constructed (CMB-S4)."
subtitle = widgets.HTML(value = f"<b><font size='2px'>{sub_text}</b>")
display(VBox([title, subtitle, s1, window]))

#display(HBox([VBox(w.children[0:13]), VBox(w.children[13:26]), VBox(list(itemgetter(-1, -2)(w.children)))]))

#display(VBox([ HBox(  VBox(w.children[0:13]),VBox(w.children[13:26])  ),      HBox(list(itemgetter(-1, -2)(w.children)))] ) )